In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import datetime
from utils.es_utils import *
from utils.env_utils import *
from utils.train_utils import *
import torch.nn.functional as F
from models.Buffer import Buffer
#from models.Generator import GeneratorPolicy
from models.DeepGenerator import GeneratorPolicy
from models.Discriminator import Discriminator
from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt
from models.Buffer import Pool
from utils.env_utils import *
from utils.eval_utils import *

In [2]:
# Training Folder
total_detections = 33
policy_dir = rf"..\data\pretuning\Pretuning - 29.08.2025_17.18"
data_path = rf'..\data\processed\pred_prey_interactions\expert_tensors\{total_detections}'
hl_path = rf'..\data\processed\pred_prey_interactions\expert_tensors\hand_labeled'
ftw_path = rf'..\data\processed\pred_prey_interactions\full_track_windows\{total_detections}'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
expert_buffer = Buffer(pred_max_length=23000, prey_max_length=73000, device=device)

# Load Expert Data from local storage
print("Expert Buffer is empty, load data...")
expert_buffer.add_expert(data_path)
expert_buffer.clear(p=80)               # Reduce ratio of non-attack data by 90%. now ~equal
expert_buffer.add_expert(hl_path)       # hand-labeled data | Pred: 1057 | Prey: 33824
expert_buffer.add_expert(hl_path)
len_exp_pred, len_exp_prey = expert_buffer.lengths()

print("Storage of Predator Expert Buffer: ", len_exp_pred)
print("Storage of Prey Expert Buffer: ", len_exp_prey, "\n")

Expert Buffer is empty, load data...
Storage of Predator Expert Buffer:  6398
Storage of Prey Expert Buffer:  73000 



In [4]:
pred_policy = torch.load(os.path.join(policy_dir, "pred_policy.pt"), weights_only=False)
prey_policy = torch.load(os.path.join(policy_dir, "prey_policy.pt"), weights_only=False)

In [5]:
# Expert Data
total_detections=33
ftw_path = rf"..\data\processed\pred_prey_interactions\full_track_windows\{total_detections}"

start_frame_pool = Pool(max_length=12100, device=device)
start_frame_pool.generate_startframes(ftw_path)
print(f"Start frames in pool: {len(start_frame_pool)}")

Start frames in pool: 11978


In [7]:
env = parallel_env(predator_count=1, prey_count=32, action_count=360, use_walls=False)
positions = start_frame_pool.sample(n=1)
env.reset(options=positions)
metrics = run_policies(env, pred_policy, prey_policy, features=5)

TypeError: run_policies() got an unexpected keyword argument 'features'